# Part 4: CATE 基础 - 每个人都不一样

## 从「平均」到「个体」：因果推断的进阶之路

之前我们学习了 ATE (Average Treatment Effect)——平均处理效应。但问题来了：

> 「平均来看有效」≠「对每个人都有效」

就像医生不能只看「这药对大多数人有效」就给每个人开同样的药一样，我们需要了解**谁会受益、谁不会受益**。

这就是 **CATE (Conditional Average Treatment Effect)** —— **条件平均处理效应**！

---

## 🎯 学习目标

1. 理解 CATE 的概念和意义
2. 识别和分析子群体效应差异
3. 可视化效应异质性
4. 基于 CATE 制定最优策略

## 🌟 生活中的 CATE：个性化医疗的例子

### 场景：一种新的降压药

假设我们在研究一种新的降压药：

- **ATE = -10 mmHg**：平均而言，吃药后血压降低 10 mmHg

但如果我们深入分析：

| 人群 | 效果 | 解读 |
|------|------|------|
| 年轻人 (20-40岁) | -5 mmHg | 效果一般 |
| 中年人 (40-60岁) | -15 mmHg | 效果显著 |
| 老年人 (60+岁) | -8 mmHg | 效果一般 |
| 有遗传因素 | -20 mmHg | 效果非常好 |
| 无遗传因素 | -3 mmHg | 几乎没效果 |

这就是**异质性处理效应 (Heterogeneous Treatment Effect, HTE)**！

### 为什么 CATE 重要？

1. **个性化决策**：知道谁受益最多，可以精准投放资源
2. **避免伤害**：有些人可能对某种处理有负面效果
3. **资源优化**：不必对所有人都采取同样的措施

## 📐 核心公式

### ATE vs CATE

$$\text{ATE} = \mathbb{E}[Y(1) - Y(0)]$$

$$\text{CATE}(x) = \mathbb{E}[Y(1) - Y(0) | X = x] = \tau(x)$$

其中：
- $Y(1)$: 接受处理时的结果
- $Y(0)$: 不接受处理时的结果
- $X$: 个体特征
- $\tau(x)$: 条件处理效应函数

### 关键关系

$$\text{ATE} = \mathbb{E}_X[\text{CATE}(X)]$$

也就是说，**ATE 是 CATE 的期望值**！

### PEHE：评估 CATE 估计质量

$$\text{PEHE} = \sqrt{\mathbb{E}[(\tau(X) - \hat{\tau}(X))^2]}$$

PEHE (Precision in Estimation of Heterogeneous Effects) 是衡量 CATE 估计精度的黄金标准。

In [ ]:
# 环境设置
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple, Dict
from sklearn.ensemble import RandomForestRegressor

# 设置随机种子
np.random.seed(42)

# 绘图设置
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('seaborn-v0_8-whitegrid')

print("环境配置完成! 📊")

## 示范 1.1: 生成异质性处理效应数据

让我们创建一个有明显**效应异质性**的数据集！

### 场景：电商促销效果

我们研究「发放优惠券」对「购买金额」的影响：
- **X1**: 用户活跃度（标准化）
- **X2**: 历史消费能力（标准化）
- **Age**: 用户年龄
- **T**: 是否发放优惠券（随机分配）
- **Y**: 购买金额

处理效应的异质性：
- 活跃度高的用户，优惠券效果更好（他们本来就爱买）
- 年轻用户对优惠券更敏感

In [ ]:
def generate_heterogeneous_treatment_data(
    n: int = 1000,
    seed: int = 42
) -> pd.DataFrame:
    """
    生成具有异质性处理效应的数据
    
    数据生成过程:
    - X1, X2 ~ N(0, 1)
    - Age ~ Uniform(20, 60)
    - T ~ Bernoulli(0.5)  # 随机分配
    - Y(0) = 10 + 2*X1 + 1*X2 + 0.1*Age + noise
    - Y(1) = Y(0) + tau(X1, Age)
    - tau(X1, Age) = 5 + 3*X1 - 0.05*Age  # 异质性效应!
    
    Returns:
        DataFrame with columns: X1, X2, Age, T, Y, tau_true
    """
    np.random.seed(seed)
    
    # 生成协变量
    X1 = np.random.randn(n)  # 用户活跃度 ~ N(0, 1)
    X2 = np.random.randn(n)  # 消费能力 ~ N(0, 1)
    Age = np.random.uniform(20, 60, n)  # 年龄 ~ Uniform(20, 60)
    
    # 随机分配处理 (50% 概率)
    T = np.random.binomial(1, 0.5, n)
    
    # 计算异质性处理效应 tau
    # tau(X1, Age) = 5 + 3*X1 - 0.05*Age
    tau_true = 5 + 3*X1 - 0.05*Age
    
    # 生成潜在结果
    noise = np.random.randn(n) * 0.5
    Y0 = 10 + 2*X1 + 1*X2 + 0.1*Age + noise  # Y(0) = 基础购买金额
    Y1 = Y0 + tau_true  # Y(1) = Y(0) + 处理效应
    
    # 计算观测结果 - 根据 T 选择 Y(1) 或 Y(0)
    Y = T * Y1 + (1 - T) * Y0
    
    return pd.DataFrame({
        'X1': X1,
        'X2': X2,
        'Age': Age,
        'T': T,
        'Y': Y,
        'tau_true': tau_true
    })

In [ ]:
# 测试数据生成
df = generate_heterogeneous_treatment_data(n=2000)

if df is not None and df['X1'].iloc[0] is not None:
    print("数据生成成功! 🎉")
    print(f"\n数据形状: {df.shape}")
    print(f"\n前 5 行:")
    display(df.head())
    
    print(f"\n=== 数据统计 ===")
    print(f"处理组比例: {df['T'].mean():.2%}")
    print(f"\n=== 真实因果效应 ===")
    print(f"真实 ATE: {df['tau_true'].mean():.4f}")
    print(f"真实 CATE 范围: [{df['tau_true'].min():.2f}, {df['tau_true'].max():.2f}]")
    print(f"真实 CATE 标准差: {df['tau_true'].std():.4f}")
    
    print(f"\n=== 朴素估计（有偏差风险）===")
    naive_ate = df.loc[df['T']==1, 'Y'].mean() - df.loc[df['T']==0, 'Y'].mean()
    print(f"朴素 ATE 估计: {naive_ate:.4f}")
    print(f"偏差: {naive_ate - df['tau_true'].mean():.4f}")
    print("(因为是随机分配，偏差应该很小)")
else:
    print("[TODO] 请完成 generate_heterogeneous_treatment_data 函数")

In [ ]:
# 可视化效应异质性
if df is not None and df['X1'].iloc[0] is not None:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # CATE 分布
    axes[0].hist(df['tau_true'], bins='auto', alpha=0.7, color='#3498db', edgecolor='black')
    axes[0].axvline(x=df['tau_true'].mean(), color='red', linestyle='--', 
                   label=f'ATE = {df["tau_true"].mean():.2f}')
    axes[0].set_xlabel('CATE (真实处理效应)')
    axes[0].set_ylabel('频数')
    axes[0].set_title('CATE 分布 - 每个人都不一样!')
    axes[0].legend()
    
    # CATE vs X1
    axes[1].scatter(df['X1'], df['tau_true'], alpha=0.3, c='#2ecc71')
    z = np.polyfit(df['X1'], df['tau_true'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(df['X1'].min(), df['X1'].max(), 100)
    axes[1].plot(x_line, p(x_line), 'r-', linewidth=2, label='趋势线')
    axes[1].set_xlabel('X1 (用户活跃度)')
    axes[1].set_ylabel('CATE')
    axes[1].set_title('CATE vs 活跃度\n活跃度越高，效应越大')
    axes[1].legend()
    
    # CATE vs Age
    axes[2].scatter(df['Age'], df['tau_true'], alpha=0.3, c='#e74c3c')
    z = np.polyfit(df['Age'], df['tau_true'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(df['Age'].min(), df['Age'].max(), 100)
    axes[2].plot(x_line, p(x_line), 'b-', linewidth=2, label='趋势线')
    axes[2].set_xlabel('Age (年龄)')
    axes[2].set_ylabel('CATE')
    axes[2].set_title('CATE vs 年龄\n年龄越大，效应越小')
    axes[2].legend()
    
    plt.tight_layout()
    plt.show()

## 示范 1.2: 使用 T-Learner 估计 CATE

T-Learner 是最简单直观的 CATE 估计方法：

1. 用控制组数据训练一个模型 $\hat{\mu}_0(x) = \mathbb{E}[Y|T=0, X=x]$
2. 用处理组数据训练另一个模型 $\hat{\mu}_1(x) = \mathbb{E}[Y|T=1, X=x]$
3. CATE 估计: $\hat{\tau}(x) = \hat{\mu}_1(x) - \hat{\mu}_0(x)$

### 直觉理解

想象你开两家「平行宇宙」的咖啡店：
- 一家店从不做促销（学习「不促销」的销量规律）
- 一家店总是做促销（学习「促销」的销量规律）

然后对任何新顾客，你都可以预测：「如果对这个人促销 vs 不促销，销量差多少？」

In [ ]:
class SimpleTLearner:
    """
    T-Learner: 分别为处理组和对照组训练模型
    
    CATE(x) = E[Y|T=1,X=x] - E[Y|T=0,X=x]
    """
    
    def __init__(self):
        self.model_0 = RandomForestRegressor(
            n_estimators=100, 
            max_depth=10,
            random_state=42
        )
        self.model_1 = RandomForestRegressor(
            n_estimators=100, 
            max_depth=10,
            random_state=43
        )
    
    def fit(self, X: np.ndarray, T: np.ndarray, Y: np.ndarray):
        """
        训练 T-Learner
        """
        # 分离处理组和对照组
        mask_0 = (T == 0)  # 对照组掩码
        mask_1 = (T == 1)  # 处理组掩码
        
        # 分别训练两个模型
        self.model_0.fit(X[mask_0], Y[mask_0])  # 在对照组上训练
        self.model_1.fit(X[mask_1], Y[mask_1])  # 在处理组上训练
        
        return self
    
    def predict_cate(self, X: np.ndarray) -> np.ndarray:
        """
        预测 CATE
        """
        # 预测 Y(0) 和 Y(1)
        Y0_pred = self.model_0.predict(X)
        Y1_pred = self.model_1.predict(X)
        
        # 计算 CATE = Y(1) - Y(0)
        cate_pred = Y1_pred - Y0_pred
        
        return cate_pred

In [ ]:
# 测试 T-Learner
if df is not None and df['X1'].iloc[0] is not None:
    # 准备数据
    X = df[['X1', 'X2', 'Age']].values
    T = df['T'].values
    Y = df['Y'].values
    tau_true = df['tau_true'].values
    
    try:
        # 训练 T-Learner
        learner = SimpleTLearner()
        learner.fit(X, T, Y)
        tau_pred = learner.predict_cate(X)
        
        if tau_pred is not None:
            print("T-Learner 训练成功! 🎉")
            print(f"\n预测 CATE 统计:")
            print(f"  均值: {tau_pred.mean():.4f} (真实: {tau_true.mean():.4f})")
            print(f"  范围: [{tau_pred.min():.2f}, {tau_pred.max():.2f}]")
        else:
            print("[TODO] 请完成 SimpleTLearner.predict_cate 方法")
    except Exception as e:
        print(f"[TODO] SimpleTLearner 实现有误: {e}")

## 示范 1.3: ATE 和 PEHE 计算

现在让我们来计算两个关键指标：

1. **ATE**: 平均处理效应（期望值）
2. **PEHE**: 异质效应估计精度（RMSE）

In [ ]:
def compute_ate(tau: np.ndarray) -> float:
    """
    计算平均处理效应 (ATE)
    
    ATE = E[tau(X)] = mean(tau)
    """
    return np.mean(tau)


def compute_pehe(tau_true: np.ndarray, tau_pred: np.ndarray) -> float:
    """
    计算 PEHE (Precision in Estimation of Heterogeneous Effects)
    
    PEHE = sqrt(E[(tau_true - tau_pred)^2])
    
    这是衡量 CATE 估计精度的黄金标准
    """
    return np.sqrt(np.mean((tau_true - tau_pred)**2))

In [ ]:
# 测试 ATE 和 PEHE
if 'tau_pred' in dir() and tau_pred is not None:
    ate_true = compute_ate(tau_true)
    ate_pred = compute_ate(tau_pred)
    pehe = compute_pehe(tau_true, tau_pred)
    
    if ate_true is not None and pehe is not None:
        print("评估指标计算成功! 🎉")
        print(f"\n=== ATE 评估 ===")
        print(f"真实 ATE: {ate_true:.4f}")
        print(f"预测 ATE: {ate_pred:.4f}")
        print(f"ATE 误差: {abs(ate_true - ate_pred):.4f}")
        
        print(f"\n=== CATE 评估 ===")
        print(f"PEHE: {pehe:.4f}")
        print(f"(PEHE 越小越好，0 表示完美预测)")
        
        # ITE 相关性
        corr = np.corrcoef(tau_true, tau_pred)[0, 1]
        print(f"\nITE 相关性: {corr:.4f}")
        print(f"(1.0 表示完美排序，0 表示随机)")
    else:
        print("[TODO] 请完成 compute_ate 和 compute_pehe 函数")

In [ ]:
# 可视化预测 vs 真实
if 'tau_pred' in dir() and tau_pred is not None and 'pehe' in dir() and pehe is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 散点图
    axes[0].scatter(tau_true, tau_pred, alpha=0.3, c='#3498db')
    min_val = min(tau_true.min(), tau_pred.min())
    max_val = max(tau_true.max(), tau_pred.max())
    axes[0].plot([min_val, max_val], [min_val, max_val], 'r--', label='完美预测')
    axes[0].set_xlabel('真实 CATE')
    axes[0].set_ylabel('预测 CATE')
    axes[0].set_title(f'CATE 预测 vs 真实\nPEHE = {pehe:.4f}')
    axes[0].legend()
    
    # 误差分布
    errors = tau_pred - tau_true
    axes[1].hist(errors, bins='auto', alpha=0.7, color='#9b59b6', edgecolor='black')
    axes[1].axvline(x=0, color='red', linestyle='--', label='零误差')
    axes[1].axvline(x=errors.mean(), color='orange', linestyle='-', 
                   label=f'均值: {errors.mean():.3f}')
    axes[1].set_xlabel('预测误差 (预测 - 真实)')
    axes[1].set_ylabel('频数')
    axes[1].set_title('CATE 预测误差分布')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()

## 示范 1.4: 子群体分析

实际应用中，我们通常想知道：**哪些人对处理反应最好？**

让我们将用户按 CATE 大小分组，分析每组的特征！

In [ ]:
def identify_subgroups_by_cate(
    tau: np.ndarray,
    n_groups: int = 4
) -> np.ndarray:
    """
    根据 CATE 大小将样本分为若干子群体
    
    使用分位数分组:
    - Q1: CATE 最低的 25%
    - Q2: CATE 在 25%-50% 之间
    - Q3: CATE 在 50%-75% 之间
    - Q4: CATE 最高的 25%
    
    Returns:
        group_labels: 每个样本的组别 (0 to n_groups-1)
    """
    # 使用分位数分组
    group_labels = np.zeros(len(tau), dtype=int)
    
    # 计算分位数边界
    quantiles = np.linspace(0, 100, n_groups + 1)
    boundaries = np.percentile(tau, quantiles)
    
    # 将每个样本分配到对应的组
    for i in range(n_groups):
        if i == n_groups - 1:
            # 最后一组包含最大值
            mask = (tau >= boundaries[i])
        else:
            mask = (tau >= boundaries[i]) & (tau < boundaries[i + 1])
        group_labels[mask] = i
    
    return group_labels

In [ ]:
def analyze_subgroup_effects(
    df: pd.DataFrame,
    group_labels: np.ndarray,
    tau_pred: np.ndarray
) -> pd.DataFrame:
    """
    分析每个子群体的处理效应
    """
    results = []
    n_groups = len(np.unique(group_labels))
    
    for g in range(n_groups):
        mask = group_labels == g
        
        # 计算该组的统计量
        group_size = mask.sum()
        true_cate = df.loc[mask, 'tau_true'].mean()
        pred_cate = tau_pred[mask].mean()
        
        # 计算观测的处理效应
        df_group = df[mask]
        treated_mask = df_group['T'] == 1
        control_mask = df_group['T'] == 0
        
        if treated_mask.sum() > 0 and control_mask.sum() > 0:
            obs_effect = df_group.loc[treated_mask, 'Y'].mean() - df_group.loc[control_mask, 'Y'].mean()
        else:
            obs_effect = np.nan
        
        # 特征均值
        avg_x1 = df.loc[mask, 'X1'].mean()
        avg_age = df.loc[mask, 'Age'].mean()
        
        results.append({
            'Group': f'Q{g+1}',
            'Size': group_size,
            'True CATE': true_cate,
            'Pred CATE': pred_cate,
            'Obs Effect': obs_effect,
            'Avg X1': avg_x1,
            'Avg Age': avg_age
        })
    
    return pd.DataFrame(results)

In [ ]:
# 子群体分析
if 'tau_pred' in dir() and tau_pred is not None:
    try:
        # 根据真实 CATE 分组
        groups = identify_subgroups_by_cate(tau_true, n_groups=4)
        
        if groups is not None:
            subgroup_df = analyze_subgroup_effects(df, groups, tau_pred)
            
            if subgroup_df is not None and subgroup_df['Size'].iloc[0] is not None:
                print("子群体分析结果: 🎯")
                print("="*80)
                display(subgroup_df.round(2))
                
                print("\n解读:")
                print("- Q1: CATE 最低的 25% (效果最差的人群)")
                print("- Q4: CATE 最高的 25% (效果最好的人群)")
                print("\n观察:")
                print(f"- Q4 用户平均 X1 = {subgroup_df.loc[3, 'Avg X1']:.2f} (更活跃)")
                print(f"- Q4 用户平均年龄 = {subgroup_df.loc[3, 'Avg Age']:.1f} (更年轻)")
            else:
                print("[TODO] 请完成 analyze_subgroup_effects 函数")
        else:
            print("[TODO] 请完成 identify_subgroups_by_cate 函数")
    except Exception as e:
        print(f"[错误] {e}")

In [ ]:
# 可视化子群体
if 'groups' in dir() and groups is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    colors = ['#e74c3c', '#f39c12', '#2ecc71', '#3498db']  # 红 -> 蓝
    
    # 子群体在特征空间中的分布
    for g in range(4):
        mask = groups == g
        axes[0].scatter(
            df.loc[mask, 'X1'], 
            df.loc[mask, 'Age'],
            alpha=0.5, 
            c=colors[g],
            label=f'Q{g+1} (n={mask.sum()})'
        )
    axes[0].set_xlabel('X1 (活跃度)')
    axes[0].set_ylabel('Age (年龄)')
    axes[0].set_title('子群体在特征空间的分布')
    axes[0].legend()
    
    # 子群体 CATE 分布
    for g in range(4):
        mask = groups == g
        axes[1].hist(
            df.loc[mask, 'tau_true'], 
            bins='auto', 
            alpha=0.6,
            color=colors[g],
            label=f'Q{g+1}'
        )
    axes[1].set_xlabel('CATE')
    axes[1].set_ylabel('频数')
    axes[1].set_title('各子群体的 CATE 分布')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()

## 示范 1.5: 基于 CATE 的最优处理策略

CATE 的终极应用：**谁应该被处理？**

如果处理有成本（比如发优惠券要花钱），我们应该只对**预期收益 > 成本**的用户进行处理。

### 最优策略

$$\text{处理决策} = \begin{cases} 1 & \text{if } \hat{\tau}(x) > \text{cost} \\ 0 & \text{otherwise} \end{cases}$$

In [ ]:
def optimal_treatment_policy(
    tau_pred: np.ndarray,
    cost: float = 1.0,
    threshold: float = None
) -> Tuple[np.ndarray, Dict]:
    """
    基于 CATE 的最优处理策略
    
    策略: 只对预测 CATE > threshold 的样本进行处理
    
    Args:
        tau_pred: 预测的 CATE
        cost: 处理成本
        threshold: CATE 阈值 (默认使用 cost)
    
    Returns:
        (treatment_decision, policy_stats)
    """
    if threshold is None:
        threshold = cost
    
    # 决策规则: 如果 tau_pred > threshold，则处理 (=1)
    treatment_decision = (tau_pred > threshold).astype(int)
    
    # 计算策略统计
    n_treated = treatment_decision.sum()
    treatment_rate = n_treated / len(tau_pred)
    
    # 被处理样本的平均预测 CATE
    if n_treated > 0:
        avg_expected_benefit = tau_pred[treatment_decision == 1].mean()
        total_expected_value = tau_pred[treatment_decision == 1].sum() - cost * n_treated
    else:
        avg_expected_benefit = 0.0
        total_expected_value = 0.0
    
    policy_stats = {
        'threshold': threshold,
        'treatment_rate': treatment_rate,
        'avg_expected_benefit': avg_expected_benefit,
        'total_expected_value': total_expected_value
    }
    
    return treatment_decision, policy_stats

In [ ]:
# 测试最优策略
if 'tau_pred' in dir() and tau_pred is not None:
    try:
        # 策略对比
        print("=== 处理策略对比 ===")
        print(f"假设处理成本 = 2.0\n")
        
        # 策略 1: 随机处理 50%
        random_value = tau_true.mean() * 0.5 - 2.0 * 0.5  # 50% 被处理
        print(f"策略 1 (随机处理 50%):")
        print(f"  预期总价值: {random_value * len(tau_true):.2f}")
        
        # 策略 2: 处理所有人
        all_value = tau_true.mean() - 2.0
        print(f"\n策略 2 (处理所有人):")
        print(f"  预期总价值: {all_value * len(tau_true):.2f}")
        
        # 策略 3: 最优策略 (基于 CATE)
        treatment, stats = optimal_treatment_policy(tau_pred, cost=2.0)
        
        if treatment is not None and stats['treatment_rate'] is not None:
            print(f"\n策略 3 (最优策略 - CATE > 成本):")
            print(f"  处理比例: {stats['treatment_rate']:.2%}")
            print(f"  被处理者平均预测 CATE: {stats['avg_expected_benefit']:.4f}")
            print(f"  预期总价值: {stats['total_expected_value']:.2f}")
            
            # 用真实 CATE 计算实际价值（模拟如果真的执行这个策略）
            actual_value = tau_true[treatment == 1].sum() - 2.0 * treatment.sum()
            print(f"  实际总价值（用真实 CATE）: {actual_value:.2f}")
            
            print(f"\n策略提升:")
            print(f"  vs 随机策略: +{(actual_value - random_value * len(tau_true)):.2f}")
            print(f"  vs 全处理策略: +{(actual_value - all_value * len(tau_true)):.2f}")
        else:
            print("[TODO] 请完成 optimal_treatment_policy 函数")
    except Exception as e:
        print(f"[错误] {e}")

In [ ]:
# 可视化不同阈值下的策略效果
if 'tau_pred' in dir() and tau_pred is not None:
    thresholds = np.linspace(-5, 10, 50)
    treatment_rates = []
    expected_values = []
    actual_values = []
    
    cost = 2.0
    
    for thresh in thresholds:
        treated = (tau_pred > thresh).astype(int)
        rate = treated.mean()
        treatment_rates.append(rate)
        
        # 预期价值
        if rate > 0:
            expected = tau_pred[treated == 1].sum() - cost * treated.sum()
            actual = tau_true[treated == 1].sum() - cost * treated.sum()
        else:
            expected = 0
            actual = 0
        expected_values.append(expected)
        actual_values.append(actual)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 处理比例 vs 阈值
    axes[0].plot(thresholds, treatment_rates, 'b-', linewidth=2)
    axes[0].axvline(x=cost, color='red', linestyle='--', label=f'成本阈值 = {cost}')
    axes[0].set_xlabel('CATE 阈值')
    axes[0].set_ylabel('处理比例')
    axes[0].set_title('处理比例随阈值变化')
    axes[0].legend()
    
    # 总价值 vs 阈值
    axes[1].plot(thresholds, actual_values, 'g-', linewidth=2, label='实际价值')
    axes[1].plot(thresholds, expected_values, 'b--', linewidth=2, alpha=0.7, label='预期价值')
    
    # 标记最优点
    best_idx = np.argmax(actual_values)
    axes[1].scatter([thresholds[best_idx]], [actual_values[best_idx]], 
                   color='red', s=100, zorder=5, label=f'最优阈值={thresholds[best_idx]:.2f}')
    
    axes[1].axhline(y=0, color='gray', linestyle='-', alpha=0.3)
    axes[1].set_xlabel('CATE 阈值')
    axes[1].set_ylabel('总价值')
    axes[1].set_title('策略总价值随阈值变化')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n最优阈值: {thresholds[best_idx]:.2f}")
    print(f"最大实际价值: {actual_values[best_idx]:.2f}")

## 🤔 思考题

在代码单元格中写下你的答案，然后对照参考答案。

## 🎯 面试题模拟 (高频考点)

### Q1: CATE 和 ATE 的关系是什么？

**标准答案**:
- ATE 是 CATE 的期望值: $\text{ATE} = \mathbb{E}_X[\text{CATE}(X)]$
- CATE 是 ATE 的条件化版本，反映了效应的异质性
- 如果 CATE 对所有 x 都相同，那么 CATE(x) = ATE (无异质性)

### Q2: 为什么需要 CATE？ATE 不够吗？

**标准答案**:
1. **个性化决策**: ATE 只告诉我们平均效果，但不同人可能反应不同
2. **资源优化**: 可以只对高 CATE 人群干预，避免浪费
3. **识别负效应**: 某些子群可能有负效应，ATE 可能掩盖这点
4. **理解机制**: CATE 揭示了"为什么对某些人有效"

### Q3: T-Learner 的优缺点是什么？

**标准答案**:

优点:
- 实现简单直观
- 允许处理组和对照组有完全不同的模型
- 无正则化偏差

缺点:
- 方差较大 (需要两倍样本)
- 样本不平衡时表现差
- 没有利用 T 和 X 的交互信息

### Q4: 什么时候 PEHE 会很大？

**标准答案**:
1. 模型选择错误 (如线性模型拟合非线性关系)
2. 重要协变量缺失
3. 异质性太强而模型容量不够
4. 过拟合或欠拟合
5. 样本量不足

### Q5: 如何在实践中验证 CATE 模型？(没有真实 CATE 时)

**标准答案**:
1. **分组 ATE 一致性**: 按预测 CATE 分组，验证每组观测 ATE 是否单调
2. **Uplift 曲线**: 使用 Qini 曲线评估排序能力
3. **Cross-validation**: 在不同数据分割上验证稳定性
4. **A/B 测试验证**: 在预测高 CATE 人群上做实验验证
5. **特征重要性**: 检查重要特征是否符合领域知识

In [ ]:
# 思考题 1: CATE 和 ATE 的关系是什么？ATE 能否反映异质性？

answer_1 = """
你的答案:



"""
print(answer_1)

<details>
<summary>💡 点击查看参考答案</summary>

**关系**:
- ATE 是 CATE 的期望值: $\text{ATE} = \mathbb{E}_X[\text{CATE}(X)]$
- CATE 是 ATE 的条件化版本，针对特定特征 X

**ATE 能否反映异质性**:
- **不能**！ATE 只是一个平均值，会掩盖个体差异
- 即使 ATE = 10，可能有人效应是 +20，有人是 0，有人是 -5
- 这就是为什么需要 CATE 来捕捉异质性

**关键洞察**: ATE 告诉我们"平均效果"，CATE 告诉我们"谁受益、谁不受益"

</details>

<details>
<summary>💡 点击查看参考答案</summary>

**CATE 恒定的条件**:

当且仅当处理效应不依赖于任何协变量 X 时，所有个体的 CATE 都相同。

数学表达:
$$\text{CATE}(x) = c \quad \forall x$$

这意味着: $\text{CATE}(x) = \text{ATE} = c$

**实际例子**:
- 如果一种药对所有人效果都一样（不管年龄、性别、基因），则无异质性
- 数据生成过程: $Y(1) = Y(0) + c$，其中 $c$ 是常数

**检验方法**:
- 检查 $\text{Var}[\text{CATE}(X)] = 0$
- 或者看 CATE 的分布是否是单点分布

</details>

<details>
<summary>💡 点击查看参考答案</summary>

**PEHE 的优势**:

1. **直接衡量异质性估计精度**
   - PEHE 衡量的是个体层面的误差: $\sqrt{\mathbb{E}[(\tau(X) - \hat{\tau}(X))^2]}$
   - 这正是 CATE 估计的核心目标

2. **与 ATE 评估的区别**
   - ATE 评估: 只关心 $|\mathbb{E}[\tau] - \mathbb{E}[\hat{\tau}]|$（均值误差）
   - PEHE 评估: 关心每个个体的预测误差（完整分布）

3. **为什么重要**
   - 即使 ATE 估计准确，CATE 估计可能完全错误
   - 例如: 真实 CATE = [0, 10, 20]，预测 CATE = [10, 10, 10]
     - ATE 都是 10（准确！）
     - 但 PEHE = $\sqrt{(100 + 0 + 100)/3} \approx 8.16$（很大！）

**关键**: PEHE 确保我们不仅知道平均效应，还能准确识别谁受益多、谁受益少

</details>

<details>
<summary>💡 点击查看参考答案</summary>

**验证子群体分析可靠性的方法**:

1. **交叉验证**
   - 在不同数据分割上重复子群体分析
   - 检查子群体特征是否稳定

2. **样本量检查**
   - 确保每个子群体有足够样本量（建议 n > 100）
   - 样本太少会导致高方差

3. **置信区间**
   - 为每个子群体的 CATE 估计计算置信区间
   - 使用 bootstrap 或其他重采样方法

4. **单调性检查**
   - 如果按预测 CATE 分组，观测效应应该单调递增
   - 如果不单调，可能是过拟合

5. **A/B 测试验证**
   - 在预测的高/低 CATE 人群上分别做实验
   - 验证预测的异质性是否真实存在

6. **领域知识验证**
   - 子群体的特征组合是否有业务解释？
   - 效应大小是否合理？

**最重要**: 始终在留出集 (holdout set) 上评估，不要在训练集上评估子群体效应！

</details>

<details>
<summary>💡 点击查看参考答案</summary>

**基于 CATE 的实际应用场景**:

1. **精准营销**
   - 识别对优惠券最敏感的用户群体
   - 只对高响应用户发放优惠，节省营销成本
   - 例如: 电商平台的个性化促销

2. **个性化医疗**
   - 根据患者基因、年龄、病史预测药物疗效
   - 为每个患者选择最有效的治疗方案
   - 例如: 癌症靶向治疗的患者筛选

3. **教育资源分配**
   - 识别对不同教学方法响应不同的学生
   - 为每个学生匹配最适合的教学方式
   - 例如: 在线教育的个性化学习路径

4. **信贷政策优化**
   - 预测不同用户对利率调整的敏感度
   - 制定个性化利率或额度策略
   - 例如: 银行的差异化定价

5. **员工激励**
   - 识别对不同激励方式响应不同的员工
   - 设计个性化的薪酬和福利方案
   - 例如: 销售提成方案优化

6. **公共政策**
   - 评估政策对不同人群的影响
   - 设计差异化的政策干预
   - 例如: 失业补贴对不同年龄段的就业影响

**共同特点**: 所有场景都涉及"选择性干预"——只对预期收益高的对象采取行动

</details>

In [ ]:
# 思考题 2: 在什么情况下，所有个体的 CATE 都相同（即没有异质性）？

answer_2 = """
你的答案:



"""
print(answer_2)

In [ ]:
# 思考题 3: 为什么 PEHE 是评估 CATE 估计的好指标？它与评估 ATE 的指标有何不同？

answer_3 = """
你的答案:



"""
print(answer_3)

In [ ]:
# 思考题 4: 在实践中，如何验证子群体分析的结果是可靠的（而非过拟合）？

answer_4 = """
你的答案:



"""
print(answer_4)

In [ ]:
# 思考题 5: 基于 CATE 的处理策略有哪些实际应用场景？

answer_5 = """
你的答案:



"""
print(answer_5)

## 📊 总结

### CATE 核心概念

| 概念 | 公式 | 含义 |
|------|------|------|
| ATE | $\mathbb{E}[Y(1) - Y(0)]$ | 所有人的平均效应 |
| CATE | $\mathbb{E}[Y(1) - Y(0) \| X=x]$ | 特定特征人群的效应 |
| ITE | $Y_i(1) - Y_i(0)$ | 个体 i 的效应 |
| PEHE | $\sqrt{\mathbb{E}[(\tau - \hat{\tau})^2]}$ | CATE 估计误差 |

### T-Learner 方法

| 步骤 | 说明 |
|------|------|
| 1 | 在 T=0 组训练 $\hat{\mu}_0(x)$ |
| 2 | 在 T=1 组训练 $\hat{\mu}_1(x)$ |
| 3 | $\hat{\tau}(x) = \hat{\mu}_1(x) - \hat{\mu}_0(x)$ |

### 最优处理策略

$$\pi^*(x) = \mathbb{1}[\hat{\tau}(x) > \text{cost}]$$

---

### 下一步

T-Learner 是最简单的方法，但有其局限性。在下一个练习中，我们将学习**因果森林 (Causal Forest)**——一种专门为估计 CATE 设计的方法！

In [ ]:
print("📊 恭喜完成 CATE 基础练习!")
print("\n你已经学会了:")
print("  ✓ CATE 的概念和与 ATE 的关系")
print("  ✓ 使用 T-Learner 估计 CATE")
print("  ✓ 计算 ATE 和 PEHE")
print("  ✓ 子群体分析")
print("  ✓ 基于 CATE 的最优策略")
print("\n下一步: 因果森林 - 更强大的 CATE 估计方法!")